In [ ]:
import os
import sys
from datetime import datetime

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve
from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from HUGS.Client import Search
from HUGS.Util import get_datapath

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.cm as cm
import xarray as xr
import numpy as np
import json

import ipyleaflet as ipl
import ipywidgets as ipw

In [ ]:
base_url= "https://hugs.acquire-aaai.com/t"
search = Search(service_url=base_url)

In [ ]:
base_url= "https://hugs.acquire-aaai.com/t"
search = Search(service_url=base_url)

search_terms = ["ch4"]
locations = []

noaa_results = search.search(search_terms=search_terms, locations=locations, data_type="NOAA")

In [ ]:
import ipyleaflet as ipl

center = [-5,37]
zoom = 2
noaa_map = ipl.Map(center=center, zoom=zoom)

noaa_map.layout.width = '50%'
noaa_map.layout.height = '400px'

# Load in the ACRG site data
acrg_json = "../site_data/acrg_site_info.json"
with open(acrg_json, "r") as f:
    acrg_sites = json.load(f)

for res in noaa_results:
    site = noaa_results[res]["metadata"]["site"]
    
    site = site.upper()
 
    species = noaa_results[res]["metadata"]["species"]
    start_date = noaa_results[res]["start_date"]
    end_date = noaa_results[res]["end_date"]
    
    # Get the latitude and longitude from the ACRG site info
    # As we don't have all the site data stored we skip a few datasets
    try:
        long = acrg_sites[site]["NOAA"]["longitude"]
        lat = acrg_sites[site]["NOAA"]["latitude"]
        long_name = acrg_sites[site]["NOAA"]["long_name"]

        marker = ipl.Marker(location=(lat, long), draggable=False)
        marker.popup = ipw.HTML(value=f"Site: {long_name} ({site})<br>Species: {species.upper()}<br>Daterange: {start_date} -<br>{end_date}")

        noaa_map.add_layer(marker)
    except:
        pass

# Now we overlay the footprint image. In the future this will be dynamically updated from NetCDF
edgar_image_path = "Emissions_Americas.png"

edgar_layer = ipl.ImageOverlay(url=edgar_image_path, bounds=((-60,-140), (55,-30)))
noaa_map.add_layer(edgar_layer)

In [ ]:
euro_search_terms = ["co2"]
euro_locations = []

eurocom_results = search.search(search_terms=euro_search_terms, locations=euro_locations, data_type="EUROCOM")


In [ ]:
center = [55, 2]
zoom = 4
eurocom_map = ipl.Map(center=center, zoom=zoom)

eurocom_map.layout.width = '50%'
eurocom_map.layout.height = '400px'

for res in eurocom_results:
    site = eurocom_results[res]["metadata"]["site"]
    
    site = site.upper()
 
    species = eurocom_results[res]["metadata"]["species"]
    start_date = eurocom_results[res]["start_date"]
    end_date = eurocom_results[res]["end_date"]
    
    # Get the latitude and longitude from the ACRG site info
    # As we don't have all the site data stored we skip a few datasets
    try:
        # Some sites may not be associated with EUROCOM in acrg_sites
        network_key = list(acrg_sites[site].keys())[0]
        
        long = acrg_sites[site][network_key]["longitude"]
        lat = acrg_sites[site][network_key]["latitude"]
        long_name = acrg_sites[site][network_key]["long_name"]

        marker = ipl.Marker(location=(lat, long), draggable=False)
        marker.popup = ipw.HTML(value=f"Site: {long_name} ({site})<br>Species: {species.upper()}<br>Daterange: {start_date} -<br>{end_date}")

        eurocom_map.add_layer(marker)
    except:
        pass


In [ ]:
center = [51.506815, -0.56]
zoom = 10

map_london = ipl.Map(center=center, zoom=zoom)
map_london.layout.width = '50%'
map_london.layout.height = '400px'

positron_layer = ipl.basemap_to_tiles(ipl.basemaps.CartoDB.Positron)

map_london.add_layer(positron_layer)


marker_legend = ipw.HTML(value="<img src='marker-icon-blue.png'>   Current site<br><img src='marker-icon-green.png'>   Future site")
marker_control = ipl.WidgetControl(widget=marker_legend, position="topright")

lghg_sites = "../site_data/lghg_sites.json"
map_london.add_control(marker_control)

with open(lghg_sites, "r") as f:
    lghg_data = json.load(f)
    

for site in lghg_data["current"]:
    curr_site = lghg_data["current"][site]
    
    lat = curr_site["latitude"]
    long = curr_site["longitude"]
    site_name = curr_site["long_name"]
    
    marker = ipl.Marker(location=(lat, long), draggable=False)
    marker.popup = ipw.HTML(value=f"Site: {site_name} ({site})")
    map_london.add_layer(marker)
    
for site in lghg_data["future"]:
    fut_site = lghg_data["future"][site]
    
    lat = fut_site["latitude"]
    long = fut_site["longitude"]
    site_name = fut_site["long_name"]
    
    # Here we want a green icon
    icon = ipl.Icon(icon_url='marker-icon-green.png', icon_size=[25, 40], icon_anchor=[12,15])
    
    marker = ipl.Marker(location=(lat, long), draggable=False, icon=icon)
    marker.popup = ipw.HTML(value=f"Site: {site_name} ({site})")
    map_london.add_layer(marker)
    
london_footprint = "high_res_london_block_inferno_50p.png"

footprint = ipl.ImageOverlay(url=london_footprint, bounds=((51.2458, -1.259), (51.7092, 0.17389)))
map_london.add_layer(footprint)

In [ ]:
box_layout = ipw.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='80%')

text_layout = ipw.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='50%')

text = "Figure 2.  Examples of a) Global Measurements locations from the NOAA \
network, b) selected nation / continental ICOS stations and c) current and \
planned urban measurement sites from the LondonGHG project, currently\
available for analysis on the HUGS platform. GHGs currently on the platform \
include CO2, CH4, N2O, halocarbons and related tracers (e.g. CO). Panel \
(a) also shows an overlay of emissions estimates from the EDGAR dataset. \
A footprint over Greater London is overlaid on panel (c). These images are \
screenshots from a Jupyter notebook hosted on the HUGS platform."

figure_text = ipw.HTML(value=text, layout=text_layout)


complete = ipw.VBox(children=[noaa_map, eurocom_map, map_london, figure_text], layout=box_layout)

In [ ]:
complete